### Multilayer Perceptron (MLP) for binary classification

A multilayer perceptron is a stack of fully connected (FC) layers
Binary classification indicates the output lives in one dimension and is treated as a success event (Yes/No or ClassA/ClassB)

In [1]:
import sys
import os.path
import time
import logging
from tqdm import tqdm_notebook as tqdm
from typing import List, Tuple
import random
import pickle

In [2]:
import tensorflow as tf
import h5py
import anndata
import scanpy
import loompy
import pandas as pd
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt

/home/vibflysleep/anaconda3/envs/TF/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [3]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [4]:
try:
    snakemake
    up = "../.."


except NameError:
    import json
    class AttrDict(dict):
        __getattr__ = dict.__getitem__
        __setattr__ = dict.__setitem__

    with open("01-MLP.json", 'r') as fh:
        snakemake = AttrDict(json.load(fh))
        up = "../"

In [6]:
loomfile = snakemake.input["loomfile"]
model_name = snakemake.params["model"]
summary_csv = snakemake.output["summary_csv"]
test_str_txt = snakemake.output["test_str_txt"]
target = snakemake.params["target"]
random_labels = snakemake.params["random_labels"]

print(loomfile)
print(target)
print(model_name)
print(not random_labels)

/1TB/Cloud/Lab/Projects/SleepSignature/workflow/results/20201224/scran/y_KCs/y_KCs_grouping-Condition.loom
Run
/1TB/Cloud/Lab/Projects/SleepSignature/sleep_classifier/results/y_KCs_Run
True


In [7]:
import sleepapp

In [8]:
tf.random.set_seed(
    2021
)

In [9]:
tf.__version__

'2.1.0'

In [10]:
scanpy.__version__

'1.6.0'

In [11]:
anndata.__version__

'0.7.5'

In [12]:
loompy.__version__

'3.0.6'

In [13]:
pd.__version__

'1.2.0'

In [14]:
np.__version__

'1.18.1'

In [15]:
scipy.__version__

'1.4.1'

In [16]:
h5py.__version__

'2.10.0'

In [17]:
# Load data 

In [18]:
adata = sleepapp.loom2anndata(loomfile)

Opening /1TB/Cloud/Lab/Projects/SleepSignature/workflow/results/20201224/scran/y_KCs/y_KCs_grouping-Condition.loom
Transposing loom dataset so rows become cells and columns become genes
Done


/home/vibflysleep/anaconda3/envs/TF/lib/python3.7/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [19]:
adata.obs.head()

,Age,CellID,ClusterID,Clusterings.0,Clusterings.1,Clusterings.2,Clusterings.3,Clusterings.4,Clusterings.5,Clusterings.6,...,Sleep_Stage,Treatment,detected,discard,nGene,nUMI,propZero,sizeFactor,sum,total
0,6-9 days,AAACCCAGTTAGGGAC-fc9729__DGRP_Mix_Sleep_Depriv...,4,3,4,3,4,4,2,0,...,ZT 8,Wake,1298.0,0,1298,2802.0,0.870135,0.483911,2802.0,2802.0
1,6-9 days,AAACCCATCCTACTGC-fc9729__DGRP_Mix_Sleep_Depriv...,4,3,4,3,4,4,2,0,...,ZT 8,Wake,1664.0,0,1664,4794.0,0.833517,0.765969,4794.0,4794.0
2,6-9 days,AAAGGATTCCTACGGG-fc9729__DGRP_Mix_Sleep_Depriv...,4,3,4,3,4,4,2,0,...,ZT 8,Wake,2362.0,0,2362,8026.0,0.763682,1.449857,8026.0,8026.0
3,6-9 days,AAAGGGCGTTTGAAAG-fc9729__DGRP_Mix_Sleep_Depriv...,4,3,4,3,4,4,2,0,...,ZT 8,Wake,1600.0,0,1600,4022.0,0.839920,0.683532,4022.0,4022.0
4,6-7 days,AAAGGGCTCCTACCGT-fc9729__DGRP_Mix_Sleep_Depriv...,4,3,4,3,4,4,2,0,...,ZT 20,Wake,1625.0,0,1625,4303.0,0.837419,0.700299,4303.0,4303.0


In [20]:
class MLModel:
    """
    Provide keras modelling
    Assume the inheriting class provides:
    
    * self.get_labels -> method returning list of labels
    * self.n_features -> attribute yielding number of features (genes)
    
    """
    
    def __init__(self):
        self._model = None
        self._le = LabelEncoder()
        
    
    def label_encoder(self, labels):
        """
        Encode str labels to integers
        """
        self._le = self._le.fit(labels)
        y = self._le.transform(labels)
        return y
    
    def get_encoding(self, codes):
        labels = self._le.inverse_transform(codes)
        mapping = {}
    
        for i in range(len(codes)):
                print(f"Label {labels[i]} = Code {codes[i]}")
                mapping[codes[i]] = labels[i]                
        return mapping
    
    def get_last_layer_activation_function(self):
        
        if self.n_classes == 1:
            return "sigmoid"
        else:
            return "softmax"
        
        
    def get_loss(self):
        
        if self.n_classes == 1:
            return "binary_crossentropy"
        else:
            return "sparse_categorical_crossentropy"
  

    def sequential_model(self, units: Tuple = (1000, 100)):
        """
        Define a MLP model using the Keras Sequential API
        
        Arguments:
            * units: Tuple where each ith element represents the number of units in the ith layer
            First number goes to first hidden layer i.e. does not count input layer
            Last layer should have 1 unit if you are performing binary classification
            Number of hidden units should decrease as we approach the last layer
        """
        # define model
        model = Sequential()
        model.add(Dense(units[0], activation='relu', kernel_initializer='he_normal', input_shape=(self.n_features,)))

        for i in range(1, len(units) - 1):
            model.add(Dense(units[i], activation='relu', kernel_initializer='he_normal'))

        model.add(Dense(self.n_classes, activation=self.get_last_layer_activation_function()))
        self._model = model
        

    @property
    def model(self):
        return self._model
    
    @model.setter
    def model(self, x):
        print("Setting model")
        self._model = x

    def set_target(self, column_name):
        self._target = column_name
    
    def compile(self):
        # compile the model
        self.model.compile(optimizer='adam', loss=self.get_loss(), metrics=['accuracy'])
        
    def fit(self, X, y, *args, **kwargs):
        # fit the model
        self.model.fit(X, y, *args, batch_size=32, verbose=3, **kwargs)
        
    def predict(self, data):
        yhat = self.model.predict(data)
        
        if self.n_classes == 1:
            yhat = np.array([1-yhat, yhat])          
        
        pred_class = np.argmax(yhat)
        class_score = np.max(yhat)
        return {pred_class: class_score}
        
        
        
    def save(self, model="model"):
        
        if self.model is None:
            return
        else:
            # serialize model to JSON
            model_json = self.model.to_json()
            with open(f"{model}.json", "w") as json_file:
                json_file.write(model_json)
            # serialize weights to HDF5
            self.model.save_weights(f"{model}.h5")
            print("Saved model to disk")
            
    def load(self, model="model"):
        # later...
        # load json and create model
        output_model_json = ""
        json_file = open(f"{model}.json", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        print(loaded_model_json)
        loaded_model = tf.keras.models.model_from_json(loaded_model_json)
        # load weights into new model
        loaded_model.load_weights(f"{model}.h5")
        print("Loaded model from disk")
        self.model = loaded_model
        self.compile()
        return None

In [21]:
class Dataset(MLModel):
    
    def __init__(self, adata, *args, **kwargs):
        self._adata = adata
        super().__init__(*args, **kwargs)
        
    
    @staticmethod
    def xtype(x):
        return type(x).__module__
    

    @property
    def X(self):
        X = self._adata.X
        #print(self.xtype(X))
        if self.xtype(X) == "numpy":
            print("Returning numpy")
            return X
        else:
            return X.toarray()

    def get_features(self) -> np.array:
        """
        Return model imput features (genes) as a table of shape
        cells x genes
        """
        return self.X
        
    def get_labels(self, encode: bool = True, random: bool = False) -> List:
        """
        Return learnable feature (label)
        This should be a column in the obs table
        that the model is postulated to be able to infer from the transcriptome
        
        Arguments:
            * encode: the labels are returned encoded, i.e. each label becomes an integer starting from 0
            * random: if True, a set of random labels is generated with number of classes equal to those in self._target
        """

        if self._target is None:
            print("Please set _target to a column name in the cell metadata (self._adata.obs)")
            return None

        labels = self._adata.obs[self._target].values.tolist()

        if encode:
                labels = self.label_encoder(labels)
        if random:
            labels = np.random.randint(0, len(np.unique(labels)), self._adata.obs.shape[0])
        
        return labels
            
    
    @property
    def n_features(self):
        return self._adata.shape[1]
    
    @property
    def n_classes(self):
        return len(self._adata.obs[self._target].unique())
            

    def train_test_split(self, test_size=0.33, *args, **kwargs):
        X = self.get_features()
        y = self.get_labels(*args, **kwargs)
        indices = self.obs["CellID"]
        X_train, X_test, y_train, y_test, index_train, index_test = train_test_split(X, y, indices, test_size=test_size)
        return {
                "train": (X_train, y_train, index_train),
                "test": (X_test, y_test, index_test)
               }

    @property
    def obs(self):
        return self._adata.obs
    
    @property
    def var(self):
        return self._adata.var

In [22]:
data=Dataset(adata)
data.set_target(target)
print(data.n_classes)
assert data.get_features().shape[0] == len(data.get_labels())

7


In [23]:
data.get_encoding([i for i in range(len(data._adata.obs[data._target].unique()))])

Label 20180419 = Code 0
Label 20191009 = Code 1
Label 20191023 = Code 2
Label 20191106 = Code 3
Label 20200902 = Code 4
Label 20200916 = Code 5
Label 20200924 = Code 6


{0: '20180419',
 1: '20191009',
 2: '20191023',
 3: '20191106',
 4: '20200902',
 5: '20200916',
 6: '20200924'}

Define architecture (how many layers, with how many neurons, connectivity between neurons, etc)

In [24]:
dataset_split = data.train_test_split(random=random_labels)

In [25]:
dataset_split["train"][1]

array([1, 3, 2, ..., 0, 2, 3])

2954 cells
1 epoch -> model has seen all cells once


Train architecture -> populate the neurons with weights that make sense i.e. correctly map the input (genes) to the output (wake/sleep)

In [26]:
data.sequential_model()
data.compile()
data.fit(*dataset_split["train"][:2], epochs=150)

Train on 2954 samples
Epoch 1/150
Epoch 2/150
Epoch 3/150
Epoch 4/150
Epoch 5/150
Epoch 6/150
Epoch 7/150
Epoch 8/150
Epoch 9/150
Epoch 10/150
Epoch 11/150
Epoch 12/150
Epoch 13/150
Epoch 14/150
Epoch 15/150
Epoch 16/150
Epoch 17/150
Epoch 18/150
Epoch 19/150
Epoch 20/150
Epoch 21/150
Epoch 22/150
Epoch 23/150
Epoch 24/150
Epoch 25/150
Epoch 26/150
Epoch 27/150
Epoch 28/150
Epoch 29/150
Epoch 30/150
Epoch 31/150
Epoch 32/150
Epoch 33/150
Epoch 34/150
Epoch 35/150
Epoch 36/150
Epoch 37/150
Epoch 38/150
Epoch 39/150
Epoch 40/150
Epoch 41/150
Epoch 42/150
Epoch 43/150
Epoch 44/150
Epoch 45/150
Epoch 46/150
Epoch 47/150
Epoch 48/150
Epoch 49/150
Epoch 50/150
Epoch 51/150
Epoch 52/150
Epoch 53/150
Epoch 54/150
Epoch 55/150
Epoch 56/150
Epoch 57/150
Epoch 58/150
Epoch 59/150
Epoch 60/150
Epoch 61/150
Epoch 62/150
Epoch 63/150
Epoch 64/150
Epoch 65/150
Epoch 66/150
Epoch 67/150
Epoch 68/150
Epoch 69/150
Epoch 70/150
Epoch 71/150
Epoch 72/150
Epoch 73/150
Epoch 74/150
Epoch 75/150
Epoch 76/150

In [25]:
# data.load(model_name)

In [26]:
data.model

In [27]:
# evaluate the model
loss, acc = data.model.evaluate(*dataset_split["test"][:2], verbose=0)
test_str = 'Test Accuracy: %.3f' % acc
print(test_str)
with open(test_str_txt, "w") as fh:
    fh.write(test_str + '\n')

Test Accuracy: 0.919


In [28]:
data.save(model_name)

Saved model to disk


In [29]:
data.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              9996000   
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 7007      
Total params: 10,003,007
Trainable params: 10,003,007
Non-trainable params: 0
_________________________________________________________________


In [30]:
np.mean(dataset_split["test"][1])

2.2479395604395602

In [42]:
#data.model.predict([dataset_split["test"][0][0,:].tolist()])
# make a prediction

yhat = data.predict([dataset_split["test"][0][0,:].tolist()])
print('Predicted score: %.3f' % list(yhat.values())[0])
print('Predicted class: %d' % list(yhat.keys())[0])
print(f"Should be {dataset_split['test'][1][0]}")

Predicted score: 1.000
Predicted class: 0
Should be 0


In [ ]:
# data.get_labels(encode=False)[:3]
# data.get_labels()[:3]

In [43]:
class_scores = []
class_preds = []

test_data = dataset_split["test"][0]

for i in tqdm(range(test_data.shape[0])):
    yhat = data.predict([test_data[i,:].tolist()])
    class_preds.append(list(yhat.keys())[0])
    class_scores.append(list(yhat.values())[0])

/home/vibflysleep/anaconda3/envs/TF/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1456 [00:00<?, ?it/s]

In [60]:
ytrue = dataset_split["test"][1]

summ = pd.DataFrame({
    "yhat": class_scores,
    "class": class_preds,
    "ytrue": ytrue,
})

In [61]:
summ["correct_pred"] =  summ.values[:,1] == summ.values[:,2]
summ["incorrect_pred"] = summ.values[:,1] != summ.values[:,2]
summ


,yhat,class,ytrue,correct_pred,incorrect_pred
0,0.999907,0,0,True,False
1,1.000000,0,0,True,False
2,0.999989,5,5,True,False
3,0.851627,2,2,True,False
4,0.825658,5,5,True,False
...,...,...,...,...,...
1451,0.999993,2,2,True,False
1452,1.000000,1,1,True,False
1453,1.000000,6,6,True,False
1454,0.347401,1,0,False,True


In [62]:
#dataset_split["test"][2]

In [63]:
test_obs = pd.merge(dataset_split["test"][2], data.obs, how="left")

In [64]:
summ=pd.concat([summ, test_obs],axis=1)

In [65]:
summ.head()

,yhat,class,ytrue,correct_pred,incorrect_pred,CellID,Age,ClusterID,Clusterings.0,Clusterings.1,...,Sleep_Stage,Treatment,detected,discard,nGene,nUMI,propZero,sizeFactor,sum,total
0,0.999907,0,0,True,False,CAGGGCTCAACAGTGG-fc9729__DGRP_Mix_Sleep_Depriv...,6-9 days,4,3,4,...,ZT 20,Sleep,1622.0,0,1622,4108.0,0.837719,0.674472,4108.0,4108.0
1,1.000000,0,0,True,False,TGAGGTTCACTTCAAG-fc9729__DGRP_Mix_Sleep_Depriv...,6-7 days,4,3,4,...,ZT 8,Wake,4525.0,1,4525,41944.0,0.547274,7.168933,41944.0,41944.0
2,0.999989,5,5,True,False,GACTCTCGTGTTGACT-ce85ae__S2_10x_160920-20200916,5-7 days,4,3,4,...,ZT 8,Wake,1856.0,0,1856,5457.0,0.814307,0.895000,5457.0,5457.0
3,0.851627,2,2,True,False,ATCACTTAGTGAACAT-b0b341__S3_10x_20191023-20191023,7 days,4,3,4,...,ZT 20,Sleep,1752.0,0,1752,5594.0,0.824712,0.814993,5594.0,5594.0
4,0.825658,5,5,True,False,AGTCTCCGTCAAATCC-ce85ae__S2_10x_160920-20200916,5-7 days,4,3,4,...,ZT 8,Sleep,658.0,1,658,1626.0,0.934167,0.189393,1626.0,1626.0


In [66]:
summ.groupby("Condition").sum()

,yhat,class,ytrue,correct_pred,incorrect_pred,Clusterings.0,Clusterings.1,Clusterings.2,Clusterings.3,Clusterings.4,...,Embeddings_Y.2,Percent_mito,detected,discard,nGene,nUMI,propZero,sizeFactor,sum,total
Condition,,,,,,,,,,,,,,,,,,,,,
ZT 14 SD,31.197010,94,96,31,1,96,128,96,128,128,...,29.591022,3.640718,58805.0,2,58805,205914.0,26.116558,29.317875,205914.0,205914.0
ZT 14 sleep,77.802381,231,237,76,3,261,316,237,316,316,...,34.708839,10.276807,145030.0,9,145030,521367.0,64.489746,74.725090,521367.0,521367.0
ZT 2 rebound 12h SD,226.487216,443,428,219,10,735,916,687,916,916,...,17.029377,25.054203,436838.0,30,436838,1795278.0,185.294342,255.774628,1795278.0,1795278.0
ZT 2 sleep drive 14h SD,144.128526,229,226,140,5,459,580,435,580,580,...,8.989573,15.420468,267025.0,18,267025,1004200.0,118.284142,141.358109,1004200.0,1004200.0
ZT 20 midline cross,88.244600,177,178,88,1,283,356,267,356,356,...,22.005627,6.570885,175916.0,12,175916,673128.0,71.399597,96.645912,673128.0,673128.0
ZT 20 sleep,287.025583,548,528,262,36,926,1192,894,1192,1192,...,100.646004,24.244793,548298.0,49,548298,2002588.0,243.142776,301.424683,2002588.0,2002588.0
ZT 20 sleep deprivation,215.223485,401,369,194,30,696,896,672,896,896,...,71.477463,21.231804,411345.0,36,411345,1459433.0,182.844925,219.431213,1459433.0,1459433.0
ZT 8 20h SD,63.190920,342,362,55,14,207,276,207,276,276,...,40.386845,6.251998,114716.0,20,114716,395232.0,57.522663,62.329391,395232.0,395232.0
ZT 8 gab,76.115061,420,438,71,10,251,324,243,324,324,...,72.935493,7.518210,129407.0,21,129407,400524.0,68.052826,63.522930,400524.0,400524.0


In [67]:
summ.groupby("Genotype").sum()

,yhat,class,ytrue,correct_pred,incorrect_pred,Clusterings.0,Clusterings.1,Clusterings.2,Clusterings.3,Clusterings.4,...,Embeddings_Y.2,Percent_mito,detected,discard,nGene,nUMI,propZero,sizeFactor,sum,total
Genotype,,,,,,,,,,,,,,,,,,,,,
line_287,155.409705,579,573,144,16,488,640,480,640,640,...,107.590340,13.756172,260098.0,15,260098,798320.0,133.977188,119.779808,798320.0,798320.0
line_303,162.372277,396,397,146,20,522,664,498,664,664,...,44.577148,15.555780,320399.0,27,320399,1309891.0,133.944077,192.866348,1309891.0,1309891.0
line_313,68.884057,68,67,69,1,234,280,210,280,280,...,24.431211,3.838529,151599.0,15,151599,621403.0,54.832516,98.771049,621403.0,621403.0
line_359,15.751792,4,0,15,1,48,64,48,64,64,...,17.128075,0.794291,31909.0,4,31909,124335.0,12.807504,20.968523,124335.0,124335.0
line_379,259.685716,655,656,244,26,850,1080,810,1080,1080,...,87.123665,26.131338,503289.0,50,503289,1864700.0,219.645920,280.476501,1864700.0,1864700.0
line_441,242.670214,524,515,229,19,776,992,744,992,992,...,110.844986,20.976332,446411.0,39,446411,1567675.0,203.336563,234.721939,1567675.0,1567675.0
line_646,109.026463,353,351,106,7,387,452,339,452,452,...,38.902752,11.826299,203800.0,20,203800,771933.0,92.609802,115.383301,771933.0,771933.0
line_88,247.688806,320,306,239,14,767,1012,759,1012,1012,...,104.420280,22.582960,468627.0,29,468627,1627937.0,206.113861,245.407028,1627937.0,1627937.0
line_892,63.732961,100,101,63,1,200,256,192,256,256,...,-9.215744,6.604895,125218.0,9,125218,527045.0,51.471935,72.548286,527045.0,527045.0


In [68]:
summ.groupby("Run").sum()

,yhat,class,ytrue,correct_pred,incorrect_pred,Clusterings.0,Clusterings.1,Clusterings.2,Clusterings.3,Clusterings.4,...,Embeddings_Y.2,Percent_mito,detected,discard,nGene,nUMI,propZero,sizeFactor,sum,total
Run,,,,,,,,,,,,,,,,,,,,,
20180419,234.614130,31,0,232,8,760,960,720,960,960,...,174.983536,10.607087,461040.0,51,461040,1645433.0,193.872940,273.382324,1645433.0,1645433.0
20191009,337.009635,370,342,329,13,1066,1368,1026,1368,1368,...,-48.366936,45.411392,624681.0,37,624681,2294930.0,279.500641,323.791565,2294930.0,2294930.0
20191023,347.025318,696,698,340,9,1119,1396,1047,1396,1396,...,74.231094,27.350485,693884.0,50,693884,2869214.0,279.576874,411.135834,2869214.0,2869214.0
20191106,155.878722,466,474,154,4,498,632,474,632,632,...,82.272377,19.257898,284178.0,14,284178,998975.0,129.567978,142.707123,998975.0,998975.0
20200902,123.954674,578,548,81,56,419,548,411,548,548,...,106.674614,9.142261,239939.0,31,239939,825073.0,112.994095,130.569183,825073.0,825073.0
20200916,158.344620,817,845,147,22,515,676,507,676,676,...,138.321213,14.019900,281692.0,23,281692,897814.0,140.816711,140.956940,897814.0,897814.0
20200924,58.847145,354,366,55,6,183,244,183,244,244,...,49.339127,5.373086,97674.0,22,97674,318336.0,51.227715,51.743824,318336.0,318336.0


In [53]:
summ.to_csv(summary_csv)

In [159]:
# summ["Condition"].value_counts()

ZT 20 sleep                271
ZT 20 sleep deprivation    230
ZT 2 rebound 12h SD        208
ZT 2 sleep drive 14h SD    167
ZT 8 wake                  155
ZT 14 sleep                 90
ZT 20 midline cross         87
ZT 8 gab                    82
ZT 8 20h SD                 81
ZT 14 SD                    45
ZT 8 wake stimulation       40
Name: Condition, dtype: int64